In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import pickle
import gsdmm
from gsdmm import MovieGroupProcess
from tqdm import tqdm

Ler o CSV criado previamente e converter tokens em listas de tokens

In [17]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\Noise_remover\tweets_processados.csv'
tweets_df = pd.read_csv(caminho_arquivo)
tweets_df.dropna(axis='columns', inplace=True)
tweets_df['tokens'] = tweets_df.tokens.apply(lambda x:  re.split('\s', x))
docs = tweets_df['tokens'].tolist()

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_13592\783247071.py:4: SyntaxWarning: invalid escape sequence '\s'
  tweets_df['tokens'] = tweets_df.tokens.apply(lambda x:  re.split('\s', x))


# Treinamento do Modelo usando o GSDMM

Agora, aplicaremos o GSDMM (Gibbs Sampling Dirichlet Mixture Model) , com os seguintes parâmetros:
- K — É o número máximo de topicos a serem encontrados
- Alpha α - Determina o quão fácil mesas pequenas são removidas
- Beta β - Controla se uma mesa é escolhida por popularidade ou similaridade. Quanto menor, mais popular
- N_iters - Número de vezes em que um Subject é realocado numa nova mesa

In [18]:
mgp = MovieGroupProcess(K=10, alpha=0.1, beta=0.1, n_iters=30)
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)
y = mgp.fit(docs, n_terms)


with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:
    pickle.dump(mgp, f)
    f.close()

<>:7: SyntaxWarning: invalid escape sequence '\A'
<>:7: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_13592\3350084293.py:7: SyntaxWarning: invalid escape sequence '\A'
  with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:


In stage 0: transferred 3416 clusters with 10 clusters populated
In stage 1: transferred 2094 clusters with 10 clusters populated
In stage 2: transferred 1586 clusters with 10 clusters populated
In stage 3: transferred 1268 clusters with 10 clusters populated
In stage 4: transferred 1029 clusters with 10 clusters populated
In stage 5: transferred 955 clusters with 10 clusters populated
In stage 6: transferred 896 clusters with 10 clusters populated
In stage 7: transferred 838 clusters with 10 clusters populated
In stage 8: transferred 811 clusters with 10 clusters populated
In stage 9: transferred 800 clusters with 10 clusters populated
In stage 10: transferred 756 clusters with 10 clusters populated
In stage 11: transferred 704 clusters with 10 clusters populated
In stage 12: transferred 703 clusters with 10 clusters populated
In stage 13: transferred 667 clusters with 10 clusters populated
In stage 14: transferred 682 clusters with 10 clusters populated
In stage 15: transferred 662 c

# Explorando os Tópicos

Definiremos algumas funções para ajudar a explorar os tópicos

In [15]:
def top_words(cluster_word_distribution, top_cluster, values):
    '''prints the top words in each cluster'''
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print('Cluster %s : %s'%(cluster,sort_dicts))
        print(' — — — — — — — — —')

def cluster_importance(mgp):
    '''returns a word-topic matrix[phi] where each value represents
    the word importance for that particular cluster;
    phi[i][w] would be the importance of word w in topic i.
    '''
    n_z_w = mgp.cluster_word_distribution
    beta, V, K = mgp.beta, mgp.vocab_size, mgp.K
    phi = [{} for i in range(K)]
    for z in range(K):
        for w in n_z_w[z]:
            phi[z][w] = (n_z_w[z][w]+beta)/(sum(n_z_w[z].values())+V*beta)
    return phi

def topic_allocation(df, docs, mgp, topic_dict):
    '''allocates all topics to each document in original dataframe,
    adding two columns for cluster number and cluster description'''
    topic_allocations = []
    for doc in tqdm(docs):
        topic_label, score = mgp.choose_best_label(doc)
        topic_allocations.append(topic_label)

    df['cluster'] = topic_allocations

    df['topic_name'] = df.cluster.apply(lambda x: get_topic_name(x, topic_dict))
    print('Complete. Number of documents with topic allocated: {}'.format(len(df)))

def get_topic_name(doc, topic_dict):
    '''returns the topic name string value from a dictionary of topics'''
    topic_desc = topic_dict[doc]
    return topic_desc

Insight estatístico

In [19]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)
print('*'*20)
# topics sorted by the number of documents they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):',top_index)
print('*'*20)
# show the top 5 words in term frequency for each cluster 
topic_indices = np.arange(start=0, stop=len(doc_count), step=1)
top_words(mgp.cluster_word_distribution, topic_indices, 5)

Number of documents per topic : [ 188   48  134  144 1161  170  920 1093   58  352]
********************
Most important clusters (by number of docs inside): [4 7 6 9 0 5 3 2 8 1]
********************
Cluster 0 : [('infecção', 178), ('urinário', 135), ('urinario', 26), ('dor', 25), ('semana', 24)]
 — — — — — — — — —
Cluster 1 : [('infecção', 42), ('atendimento', 34), ('pré', 32), ('hospitalar', 31), ('urinário', 25)]
 — — — — — — — — —
Cluster 2 : [('infecção', 135), ('urinário', 101), ('mulher', 30), ('urinár', 29), ('ter', 29)]
 — — — — — — — — —
Cluster 3 : [('infecção', 139), ('urinário', 110), ('gato', 32), ('ano', 21), ('urinár', 20)]
 — — — — — — — — —
Cluster 4 : [('infecção', 1104), ('urinário', 852), ('água', 259), ('xixi', 225), ('urinario', 219)]
 — — — — — — — — —
Cluster 5 : [('infecção', 172), ('urinário', 162), ('gil', 119), ('preto', 90), ('internar', 87)]
 — — — — — — — — —
Cluster 6 : [('infecção', 982), ('urinário', 747), ('exame', 237), ('médico', 211), ('ter', 157)

Se quisermos ver a importância de algumas palavras , usando a função cluster_importance

Agora, decidimos o que é  cada tópico

In [21]:
topic_dict = {}
topic_names = ['Cluster 0,',
                'Cluster 1,',
                'Cluster 2,',
                'Cluster 3,',
                'Cluster 4,',
                'Cluster 5,',
                'Cluster 6,',
                'Cluster 7,',
                'Cluster 8,',
                'Cluster 9,']

for i, topic_num in enumerate(topic_indices):
 topic_dict[topic_num]=topic_names[i]



topic_allocation(tweets_df, docs, mgp, topic_dict)

100%|██████████| 4268/4268 [00:01<00:00, 3300.27it/s]

Complete. Number of documents with topic allocated: 4268


In [ ]:
tweets_df.to_excel('sttm_10topics_results.xlsx', index = False,  header=True)